In [1]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt

import os

In [2]:
sg.__version__

'1.2.1'

In [3]:
mydf = pd.read_csv("/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/Chronic Pancreatitis/318_1.csv")

In [4]:
mydf['weight'] = [abs(x) for x in mydf['weight']]

In [5]:
mydf

,source,target,weight,type
0,T-reg,Tumor,2.424922,Depletion
1,Other,Tumor,1.869055,Depletion
2,Helper T-Cell,Tumor,1.469400,Depletion
3,APC,Tumor,1.286881,Depletion
4,CTL,Other,1.051024,Depletion
5,Other,T-reg,0.783806,Depletion
6,CTL,Tumor,0.535342,Depletion
7,APC,APC,2.403178,Enrichment
8,Helper T-Cell,Helper T-Cell,2.217591,Enrichment
9,APC,Helper T-Cell,1.538010,Enrichment


In [6]:
weighted_features = StellarGraph(edges=mydf, edge_type_column='type')

In [7]:
print(weighted_features.info())

StellarGraph: Undirected multigraph
 Nodes: 6, Edges: 14

 Node types:
  default: [6]
    Features: none
    Edge types: default-Depletion->default, default-Enrichment->default

 Edge types:
    default-Enrichment->default: [7]
        Weights: range=[0.240492, 2.40318], mean=1.2206, std=0.898056
        Features: none
    default-Depletion->default: [7]
        Weights: range=[0.535342, 2.42492], mean=1.34578, std=0.647639
        Features: none


In [8]:
file_dirs = ["/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/" + x for x in ['Chronic Pancreatitis', 'IPMN', 'MCN', 'PanIN', 'PDAC']]

In [9]:
file_dirs

['/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/Chronic Pancreatitis',
 '/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/IPMN',
 '/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/MCN',
 '/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/PanIN',
 '/Users/morganoneka/Box/My Stuff/GraphClassification/GiottoOutput/PDAC']

In [10]:
idx = [len(os.listdir(x)) for x in file_dirs]

In [11]:
idx

[110, 176, 34, 82, 230]

In [12]:
def get_graphs(dirname):
    my_graphs=[]
    files = os.listdir(dirname)
    csvs = [x for x in files if x.endswith(".csv")]
    for f in csvs:
        mydf = pd.read_csv(dirname + "/" + f)
        mydf['weight'] = [abs(x) for x in mydf['weight']]
        # 1 = tumor
        # 2 = immune 
        # 3 = other
#         node_data = pd.DataFrame(
#             {"celltype": ["1", "2", "2", "2", "2", "3"]}, index=["Tumor", "APC", "Helper T-Cell", "CTL", "T-reg", "Other"]
#         )
        node_data = pd.DataFrame(
            {"celltype": ["1", "2", "3", "4", "5", "6"]}, index=["Tumor", "APC", "Helper T-Cell", "CTL", "T-reg", "Other"]
        )
        weighted_features = StellarGraph(nodes=node_data, edges=mydf, edge_type_column='type')
        my_graphs.append(weighted_features)
    return(my_graphs)

In [13]:
cp_graphs = get_graphs(file_dirs[0])
ipmn_graphs = get_graphs(file_dirs[1])
mcn_graphs = get_graphs(file_dirs[2])
panin_graphs = get_graphs(file_dirs[3])
pdac_graphs = get_graphs(file_dirs[4])

In [14]:
# def remove_problematic_graphs(grp):
#     return ([g for g in grp if g.number_of_edges() > 0])

# print(len(cp_graphs))
# print(len(remove_problematic_graphs(cp_graphs)))

# print(len(ipmn_graphs))
# print(len(remove_problematic_graphs(ipmn_graphs)))

# print(len(mcn_graphs))
# print(len(remove_problematic_graphs(mcn_graphs)))

# print(len(panin_graphs))
# print(len(remove_problematic_graphs(panin_graphs)))

# print(len(pdac_graphs))
# print(len(remove_problematic_graphs(pdac_graphs)))

In [15]:
all_graphs = cp_graphs + ipmn_graphs + mcn_graphs + panin_graphs + pdac_graphs
# all_graphs = cp_graphs + ipmn_graphs  + panin_graphs + pdac_graphs

In [16]:
all_graphs[0].info()

'StellarGraph: Undirected multigraph\n Nodes: 6, Edges: 13\n\n Node types:\n  default: [6]\n    Features: float32 vector, length 1\n    Edge types: default-Depletion->default, default-Enrichment->default\n\n Edge types:\n    default-Enrichment->default: [7]\n        Weights: range=[0.475616, 1.93021], mean=1.17158, std=0.644993\n        Features: none\n    default-Depletion->default: [6]\n        Weights: range=[0.136256, 4.2493], mean=2.49698, std=1.54048\n        Features: none'

In [17]:
# labels = ["CP" for x in range(idx[0])] + ["IPMN" for x in range(idx[1])] + ["MCN" for x in range(idx[2])] + ["PanIN" for x in range(idx[3])] + ["PDAC" for x in range(idx[4])]
# labels = [0 for x in range(len(cp_graphs))] + [1 for x in range(len(ipmn_graphs))] + [2 for x in range(len(mcn_graphs))] + [3 for x in range(len(panin_graphs))] + [4 for x in range(len(pdac_graphs))]

In [18]:
# CP = non-cancer
# IPMN = pre-cancer
# MCN = pre-cancer
# PanIN = pre-cancer
# PDAC = cancer
labels = [0 for x in range(len(cp_graphs))] + [1 for x in range(len(ipmn_graphs))] + [1 for x in range(len(mcn_graphs))] + [1 for x in range(len(panin_graphs))] + [2 for x in range(len(pdac_graphs))]

In [19]:
graph_labels = pd.Series(labels)
print(graph_labels)

0      0
1      0
2      0
3      0
4      0
      ..
311    2
312    2
313    2
314    2
315    2
Length: 316, dtype: int64


In [20]:
len(all_graphs)

316

In [21]:
generator = PaddedGraphGenerator(graphs=all_graphs)

In [22]:
def create_graph_classification_model(generator):
    gc_model = GCNSupervisedGraphClassification(
        layer_sizes=[64, 64],
        activations=["relu", "relu"],
        generator=generator,
        dropout=0.5,
    )
    x_inp, x_out = gc_model.in_out_tensors()
    predictions = Dense(units=32, activation="relu")(x_out)
    predictions = Dense(units=16, activation="relu")(predictions)
    predictions = Dense(units=1, activation="sigmoid")(predictions)

    # Let's create the Keras model and prepare it for training
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"])

    return model

In [23]:
epochs = 200  # maximum number of training epochs
folds = 10  # the number of folds for k-fold cross validation (default was 10)
n_repeats = 5  # the number of repeats for repeated k-fold cross validation (default was 5)
es = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=25, restore_best_weights=True
)

In [24]:
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=0, callbacks=[es],
    )
    # calculate performance on the test data and return along with history
    test_metrics = model.evaluate(test_gen, verbose=0)
    test_acc = test_metrics[model.metrics_names.index("acc")]

    return history, test_acc

def get_generators(train_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size
    )

    return train_gen, test_gen



In [25]:
test_accs = []

stratified_folds = model_selection.RepeatedStratifiedKFold(
    n_splits=folds, n_repeats=n_repeats
).split(graph_labels, graph_labels)

for i, (train_index, test_index) in enumerate(stratified_folds):
    print(f"Training and evaluating on fold {i+1} out of {folds * n_repeats}...")
    train_gen, test_gen = get_generators(
        train_index, test_index, graph_labels, batch_size=30
    )

    model = create_graph_classification_model(generator)

    history, acc = train_fold(model, train_gen, test_gen, es, epochs)

    test_accs.append(acc)

Training and evaluating on fold 1 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 2 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 3 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 4 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 5 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 6 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 7 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating on fold 8 out of 50...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Training and evaluating 

In [26]:
test_accs

[0.46875,
 0.4375,
 0.4375,
 0.4375,
 0.4375,
 0.46875,
 0.48387095,
 0.48387095,
 0.48387095,
 0.48387095,
 0.46875,
 0.4375,
 0.4375,
 0.4375,
 0.4375,
 0.46875,
 0.48387095,
 0.48387095,
 0.48387095,
 0.48387095,
 0.46875,
 0.4375,
 0.4375,
 0.4375,
 0.4375,
 0.46875,
 0.48387095,
 0.48387095,
 0.48387095,
 0.48387095,
 0.46875,
 0.4375,
 0.4375,
 0.4375,
 0.4375,
 0.46875,
 0.48387095,
 0.48387095,
 0.48387095,
 0.48387095,
 0.46875,
 0.4375,
 0.4375,
 0.4375,
 0.4375,
 0.46875,
 0.48387095,
 0.48387095,
 0.48387095,
 0.48387095]